In [1]:
import json

In [2]:
with open('nl2cyhper_data_test.txt','r',encoding='utf8') as f:
    text_data = f.read().split('\n')
    all_data = []
    for text in text_data:
        if(len(text)<=3):
            continue
        all_data.append(eval(text))
all_data[0][0]

{'question': '第一天白天拉了4-5次后来每天拉2-3次',
 'answer': '问题: 第一天白天拉了4-5次后来每天拉2-3次\n\n查询语句: 1 急性腹泻 疾病简介, 2 急性腹泻 疾病使用药品 药品, 2 急性腹泻 治疗的方法 治疗方法, 2 急性腹泻 疾病宜吃食物 食物\n\n解释: 根据用户描述的症状，白天拉肚子多次，随后每天仍有2-3次，这很可能是急性腹泻的症状。首先，需要向用户介绍急性腹泻的相关信息（疾病简介），然后提供可能使用的药品信息（疾病使用药品 药品），接着告诉用户可以采取的治疗方法（治疗的方法 治疗方法），最后提供适宜食用的食物建议（疾病宜吃食物 食物），帮助用户更好地恢复健康。由于用户没有提及具体的疾病名称，这里假设是急性腹泻，并根据这一假设生成相应的查询语句。'}

In [3]:
def append_data_to_json(data, filename):
    with open(filename, 'a', encoding='utf-8') as file:  # 使用UTF-8编码打开文件
        json_data = json.dumps(data, ensure_ascii=False)  # 将字典转换为JSON字符串，保持中文等字符的正确显示
        file.write(json_data + ',\n')  # 写入JSON数据，并在末尾添加逗号和换行符

In [4]:
file_data = []
for data in all_data:
    data = data[0]
    question,answer = data['question'],data['answer']
    if '查询语句:' not in answer or '解释:' not in answer:
        continue
    prompt = f"""提示:目前，我的图数据库中有8类实体：疾病、药品、药品商、疾病症状、食物、检查项目、治疗方法、科目。在这里面，“疾病”这个实体有7个属性：名称、治愈概率、治疗周期、疾病易感人群、疾病病因、疾病简介、预防措施；其余实体均只有名称这个属性。在实体之间，具有8个关系：疾病使用药品、治疗的方法、疾病宜吃食物、疾病忌吃食物、疾病的症状、疾病所需检查、疾病所属科目、疾病并发疾病。其中“疾病使用药品”是“疾病”和“药品”之间的关系，“治疗的方法”是“疾病”和“治疗方法”之间的关系，“疾病宜吃食物”是“疾病”和“食物”之间的关系，“疾病忌吃食物”是“疾病”和“食物”之间的关系，“疾病的症状”是“疾病”和“疾病症状”之间的关系，“疾病所需检查”是“疾病”和“检查项目”之间的关系，“疾病所属科目”是“疾病”和“科目”之间的关系，“疾病并发疾病”是“疾病”和“疾病”之间的关系。
    
现在，你是一个机器人医生，用户对你输入问题，你需要精准的理解问题的内容，根据其含义构建Neo4j数据库的查询语句，多条查询语句请用逗号分割。在输出完查询语句后，请你输出你的解释，讲清楚为什么要这些查询。
    
请注意，你的查询语句只限于两个操作：1.查询疾病的属性，你需要提供疾病具体名称和属性名。2.查询疾病与其他实体之间的关系，你需要提供疾病具体名称、关系名称、实体类别名。为了简化输出，你输出查询语句格式的应为：对于类型1的查询，输出“1 疾病名称 属性”；对于类型2的查询，输出“2 疾病名称 关系 实体类别”。如果有多个查询，每个查询之间用中文逗号隔开。在输出完查询语句后，请你输出你的解释，讲清楚为什么要这些查询。
    
用户问题：{question}
查询语句:
解释:
    """
    
    
    ans = answer[answer.index("查询语句"):]
    
    peft_one_data = {
            "conversations":[
                {
                    'role': 'user',
                    'content': prompt
                },
                {
                    'role': 'assistant',
                    'content':ans, 
                },
            ]
        }
   
    file_data.append(peft_one_data)
formatted_string = json.dumps(file_data, ensure_ascii=False, indent=2)
formatted_string_dev = json.dumps(file_data[:100], ensure_ascii=False, indent=2)
with open('peft_data.txt', 'w', encoding='utf-8') as file:
    file.write(formatted_string)
with open('dev.txt', 'w', encoding='utf-8') as file:
    file.write(formatted_string_dev)

In [5]:
#研究一下分割：
text = """
查询语句: 2 抗过敏 治疗的方法 治疗方法,2 抗过敏 疾病使用药品 药品\n\n解释: 用户提到了“抗过敏针”和“吃了1个多月的药”，暗示着用户正在对抗某种过敏性疾病。用户询问是否有更好的药物治疗方案，以及治疗所需的时间，这表明用户希望了解更有效的治疗方法以及相应的药品。因此，这里我生成了查询语句去查询与“抗过敏”相关的“治疗方法”和“药品”，以便提供可能的替代治疗方案和有效药物的信息。由于用户没有明确提及具体的疾病名称，我使用了“抗过敏”作为一个通用的疾病指代。如果用户有具体疾病的名称，应替换“抗过敏”以获取更精确的结果。
"""
text = text[text.index("查询语句")+5:]
text = text[:text.index("解释")]
text = text.split(',')
cyhper = [te.strip() for te in text]
# print(cyhper)
# assert text[0] == '1' or text[0] == '2'
for cyh in cyhper:
    cyh = cyh.split(' ')
    if cyh[0]!='1' and cyh[0]!='2':
        continue
    if cyh[0]=='1' and len(cyh)==3:
        pass
    elif cyh[0]=='2' and len(cyh)==4:
        pass
    else:
        continue

    if cyh[0]=='1':
        sql_q = "match (a:疾病{名称:'%s'}) return a.%s" % (cyh[1],cyh[2])
    else:
        sql_q = "match (a:疾病{名称:'%s'})-[r:%s]->(b:%s) return b.名称" % (cyh[1],cyh[2],cyh[3])
    print(sql_q)

match (a:疾病{名称:'抗过敏'})-[r:治疗的方法]->(b:治疗方法) return b.名称
match (a:疾病{名称:'抗过敏'})-[r:疾病使用药品]->(b:药品) return b.名称


In [6]:
4

4